In [2]:
# import load_dotenv
from dotenv import load_dotenv

# load env
load_dotenv()

True

In [5]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)

In [6]:
model = ChatOpenAI()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful chatbot capable of performing a variety of tasks on the data available"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

In [7]:
print(memory.load_memory_variables({}))

{'chat_history': []}


In [8]:
chain_input = RunnablePassthrough.assign(
    #chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")
    **{
        'chat_history': RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")
    }
)
chain = chain_input | prompt | model

In [9]:
inputs = {"input": "hi im bob"}
response = chain.invoke(inputs)
print(type(response))
print(response)

<class 'langchain_core.messages.ai.AIMessage'>
content='Hello Bob! How can I assist you today?' response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 31, 'total_tokens': 41}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-be2ca5b5-cf17-4cf9-a79a-222cb0d626eb-0'


In [10]:
print(type(memory.chat_memory))
print(memory.chat_memory)

<class 'langchain_core.chat_history.InMemoryChatMessageHistory'>



In [12]:
memory.chat_memory.add_user_message("hi im bob")
memory.chat_memory.add_ai_message('Hello Bob! How can I assist you today?')

In [13]:
inputs = {"input": "what is my name"}
response = chain.invoke(inputs)
print(response)

content='Your name is Bob. How can I help you today, Bob?' response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 53, 'total_tokens': 67}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-bbf0de06-d697-4c25-bb40-d8e44ecf1352-0'


In [15]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_history = ChatMessageHistory()
chat_history.add_user_message("hi im bob")
chat_history.add_ai_message('Hello Bob! How can I assist you today?')

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True, chat_memory=chat_history)

print(memory.load_memory_variables({}))
# {'history': [HumanMessage(content='hi im bob'), AIMessage(content='Hello Bob! How can I assist you today?')]}

chain_input =  RunnablePassthrough.assign(
    history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
)
chain =  chain_input | prompt | model

inputs = {"input": "what is my name"}
response = chain.invoke(inputs)

print(response)

{'history': [HumanMessage(content='hi im bob'), AIMessage(content='Hello Bob! How can I assist you today?')]}
content='Your name is Bob. How can I help you today, Bob?' response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 42, 'total_tokens': 56}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-afcdc9c5-adc6-4929-8279-5a15fd416075-0'
